In [6]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [7]:
# Adapter Layer

def read_csv_to_df(bucket,key):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
    
def write_df_to_s3(s3,trg_bucket,df_all,key):
    out_buffer = BytesIO()
    df_all.to_parquet(out_buffer, index=False)
    bucket_target = s3.Bucket(trg_bucket)
    bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)
    return bucket_target
    
def return_objects(bucket,key,src_format,arg_date):
    arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)
    objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= arg_date_dt]
    return objects

In [8]:
# Application Layer

def extract(key,objects,bucket):
    df_all = pd.concat([read_csv_to_df(bucket,obj.key) for obj in objects], ignore_index=True)
    
    return df_all

def transform_report(df_all,arg_date,columns):
    
    df_all = df_all.loc[:, columns]
    df_all.dropna(inplace=True)
    
    #get opening price per ISIN and Day
    df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('first')
    
    #Get closing price per ISIN and Day
    df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN', 'Date'])['StartPrice'].transform('last')
    
    #Agregation
    df_all = df_all.groupby(['ISIN', 'Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'), daily_traded_volume=('TradedVolume', 'sum'))
   
    #Percent Change Prev Closing
    df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df_all['change_prev_closing_%'] = (df_all['closing_price_eur'] - df_all['prev_closing_price']) / df_all['prev_closing_price'] * 100
    
    df_all.drop(columns=['prev_closing_price'], inplace=True)
    df_all = df_all.round(decimals=2)
    df_all = df_all[df_all.Date >= arg_date]
    
    print("Esto es el df_all dentro de Transform", df_all)
    return df_all

def load(s3,trg_bucket,df_all,key):

    listObj_key= []     
    for obj in bucket_target.objects.all():
        print(obj.key)
        listObj_key.append(obj.key)
        
    last_Key = listObj_key[-1]
    
    prq_obj = bucket_target.Object(key=last_Key).get().get('Body').read()
    data = BytesIO(prq_obj) 
   
    return data
    

def etl_report(key,columns,objects,bucket,arg_date,s3,trg_bucket):
    #extraer ,transormar, cargar/load
    
    df_all = extract(key,objects,bucket)

    df_all = transform_report(df_all,arg_date,columns)
  
    data = load(s3,trg_bucket,df_all,key)
    df_report = pd.read_parquet(data)
    print("Esto es el df report",df_report)
    return df_report

In [9]:
# main function entrypoint

def main():
    # Parameters/Configurations    
    arg_date = ''
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'xetra-bucket-rizo1'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'
    
    # Init
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(src_bucket)
    
    # run application
    arg_date= input("Ingresa una fecha yyyy-mm-dd:")  
    objects= return_objects(bucket,key,src_format,arg_date)

    etl_report(key,columns,objects,bucket,arg_date,s3,trg_bucket)
    

In [10]:
# run

main()

Ingresa una fecha yyyy-mm-dd:2022-03-11
el df con otras cosas               ISIN        Date  opening_price_eur  closing_price_eur  \
0     AT000000STR1  2022-03-10              35.85              35.85   
1     AT000000STR1  2022-03-11              35.80              35.45   
2     AT00000FACC2  2022-03-10               7.41               7.35   
3     AT00000FACC2  2022-03-11               7.49               7.84   
4     AT0000606306  2022-03-10              14.13              13.11   
...            ...         ...                ...                ...   
6424  XS2427474023  2022-03-11              23.15              23.45   
6425  XS2434891219  2022-03-10               3.55               3.55   
6426  XS2434891219  2022-03-11               3.56               3.53   
6427  XS2437455608  2022-03-10              28.92              28.92   
6428  XS2437455608  2022-03-11              25.45              25.45   

      minimum_price_eur  maximum_price_eur daily_traded_volume  \
0      